在选择了架构并设置了超参数后，我们就进入了训练阶段。 此时，我们的目标是找到使损失函数最小化的模型参数值。 经过训练后，我们将需要使用这些参数来做出未来的预测。 此外，有时我们希望提取参数，以便在其他环境中复用它们， 将模型保存下来，以便它可以在其他软件中执行， 或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作， 而忽略了操作参数的具体细节。 本节，我们将介绍以下内容：

访问参数，用于调试、诊断和可视化。

参数初始化。

在不同模型组件间共享参数。

In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.2090],
        [-0.0857]], grad_fn=<AddmmBackward>)

### 5.2.1 参数访问
我们从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 这就像模型是一个列表一样，每层的参数都在其属性中。 如下所示，我们可以检查第二个全连接层的参数。

In [3]:
print(len(net))

3


In [10]:
print(net[1])
print(net[1].state_dict())

ReLU()
OrderedDict()


In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1913,  0.0583, -0.1535, -0.0276,  0.0600,  0.0309, -0.2213,  0.3255]])), ('bias', tensor([-0.1626]))])


输出的结果告诉我们两件事：首先
1. 这个全连接层包括两个参数，分别是该层的权重和偏执。
2. 两者都为单精度浮点数float32。

注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此

#### 5.2.1.1 目标参数
注意，每个参数都表示为参数类的一个实例。 要对参数执行任何操作，首先我们需要访问底层的数值。 有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。 下面的代码从第二个全连接层（即第三个神经网络层）提取偏置， 提取后返回的是一个参数类实例，并进一步访问该参数的值

In [13]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1626], requires_grad=True)
tensor([-0.1626])


参数parameter是复合的对象，包括值、梯度和额外信息。
除了值之外，我们还可以访问每个参数的梯度。 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态

In [14]:
net[2].weight.grad == None

True

#### 5.2.1.2 一次性访问所有参数
当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。 当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂， 因为我们需要递归整个树来提取每个子块的参数。 下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。

In [16]:
print(
    *[(name, param.shape) for name, param in net[0].named_parameters()]
)
print(
    *[(name, param.shape) for name, param in net.named_parameters()]
)

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
# 或者
net.state_dict()['2.bias'].data

tensor([-0.1626])

#### 5.2.1.3 从嵌套块收集参数
让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。 我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [18]:
def block1():
    return nn.Sequential(
        nn.Linear(4,8),nn.ReLU(),
        nn.Linear(8,4),nn.ReLU()
    )

In [19]:
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(
            f'block{i}',block1()
        )
    return net

In [20]:
rgnet = nn.Sequential(
    block2(),
    nn.Linear(4, 1)
)
rgnet(X)

tensor([[-0.2205],
        [-0.2205]], grad_fn=<AddmmBackward>)

In [21]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [22]:
rgnet[0][1][0].bias.data

tensor([-0.4859, -0.0395, -0.0913, -0.2113, -0.4229, -0.4322, -0.4595, -0.3923])

### 5.2.2 参数初始化
默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。 PyTorch的nn.init模块提供了多种预置初始化方法。

#### 5.2.2.1 内置初始化
让我们首先调用内置的初始化器。 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0。

In [23]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 7.0961e-05, -5.8193e-03, -7.9188e-04,  1.5709e-02]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。

In [24]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以对某些块应用不同的初始化方法。 例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。

In [26]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-8.6754e-02,  4.3311e-01,  3.2202e-01, -3.5308e-01],
        [ 3.2252e-04, -1.1910e-01, -3.2251e-02,  6.6349e-01],
        [ 5.3471e-01,  4.4536e-01,  6.0705e-01,  5.0049e-01],
        [-5.9215e-01, -2.4174e-01,  1.7424e-01,  6.6311e-01],
        [-3.9770e-01, -5.0385e-01, -2.8655e-01,  2.4050e-02],
        [ 2.3104e-01, -6.1362e-01,  5.8659e-01,  2.2526e-01],
        [ 5.4193e-01, -6.4703e-02,  4.0796e-01, -2.6305e-01],
        [ 3.9223e-01, -7.9630e-02, -8.7609e-02, -3.2984e-01]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 5.2.2.2 自定义初始化
有时，深度学习框架没有提供我们需要的初始化方法。 在下面的例子中，我们使用以下的分布为任意权重参数\(w\)定义初始化方法：
$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

同样，我们实现了一个my_init函数来应用到net

In [30]:
def my_init(m):
    if type(m) == nn.Linear:
        print('Init ', *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init  weight torch.Size([8, 4])
Init  weight torch.Size([1, 8])


tensor([[-5.3401, -0.0000,  8.9276, -0.0000],
        [ 0.0000, -0.0000, -0.0000, -8.8434]], grad_fn=<SliceBackward>)

### 5.2.3 参数绑定
有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [31]:
# 给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,# 2
    nn.ReLU(),
    shared,#4
    nn.ReLU(),
    nn.Linear(8,1)
)
net(X)

tensor([[-0.2769],
        [-0.2790]], grad_fn=<AddmmBackward>)

In [33]:
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
# 确保它们实际上是同一个对象，而不只是有相同的值
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net[2].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
tensor([100.0000,  -0.1363,  -0.2953,   0.3320,   0.1017,   0.3108,   0.1400,
          0.2068])


上面这个例子表明，第三层和第5层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。 因此，如果我们改变其中一个参数，另一个参数也会改变。

 你可能会思考：当参数绑定时，梯度会发生什么情况？

##### 答案是由于模型参数包含梯度，
##### 因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的
##### 梯度会**加**在一起。

### 补充：共享参数的好处：
+ 共享参数可以节省内存
+ 对于CNN，共享参数使网络能够在图像的任何地方而不是仅仅在某个区域中计算
+ 对于RNN，**在各个时间步之间共享参数**，可以很好的推广到不同序列长度的示例
+ 对于自动编码器，编码器和解码器共享参数。在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交